In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.decomposition import PCA

#Colab use
#from google.colab import files
#uploaded = files.upload()
#import io

In [2]:
#standardization
def standard_scaler(X_train, X_test):
    
    scaler = StandardScaler()
    
    X_train = scaler.fit_transform(X_train) #standardize based on train data
    X_test = scaler.transform(X_test) #standardize test data
    
    filename = '../Dashboard UI/Dashboard UI/model/std_scaler.pkl'
    pickle.dump(scaler, open(filename, 'wb')) #save standardization scaler
    
    #Colab use
    #filename = 'std_scaler.pkl'
    #pickle.dump(scaler, open(filename, 'wb')) #save standardization scaler
    #files.download('std_scaler.pkl')
    
    return X_train, X_test

In [3]:
def evaluate(y_true, y_pred, label='test'):
    mse = mean_squared_error(y_true, y_pred) #calculate MSE
    rmse = np.sqrt(mse) #calculate RMSE
    variance = r2_score(y_true, y_pred) #calculate R2
    print('{} set RMSE:{}, R2:{}'.format(label, rmse, variance))

# Load Data

In [4]:
index_names = ['ID', 'Cycle']
setting_names = ['OpSet1', 'OpSet2', 'OpSet3']
sensor_names = ['SensorMeasure{}'.format(i) for i in range(1,22)] 
col_names = index_names + setting_names + sensor_names

train_data = pd.read_csv("../Data/train_set.csv")
test_data = pd.read_csv("../Data/test_set.csv")
true_RUL = pd.read_csv("../Data/RUL_FD001.txt", sep='\s+', header = None)

#Colab use
#train_data = pd.read_csv(io.BytesIO(uploaded['train_set.csv']))
#test_data = pd.read_csv(io.BytesIO(uploaded['test_set.csv']))
#true_RUL = pd.read_csv(io.BytesIO(uploaded['RUL_FD001.txt']), sep='\s+', header = None)

train_RUL = train_data['RUL']
train_RUL = train_RUL.clip(upper = 125) #clip maximum cycle at 125
test_RUL = test_data['RUL']

train_data = train_data.drop(['RUL'], 1)
test_data = test_data.drop(['RUL'], 1)

test_data = test_data.groupby(['ID'])
test_data = test_data.tail(1)

#assign to new variable for easy understanding
train = train_data
train_y = train_RUL
test = test_data.groupby(['ID']).tail(1) #get the last record for each engine
test_y = true_RUL

#only sensor value considered
train = train[sensor_names]
test = test[sensor_names]

# Feature Extraction

In [5]:
#Principle Component Analysis
def pca(X_train, X_test, y_train, y_test):
    
    #for reproducible result
    np.random.seed(2)
    
    X_train, X_test = standard_scaler(X_train, X_test) #standardize data
    
    #rename columns after standardization
    sensor_names = ['SensorMeasure{}'.format(i) for i in range(1,22)] 
    X_train = pd.DataFrame(X_train, columns = sensor_names).reset_index(drop = True)
    X_test = pd.DataFrame(X_test, columns = sensor_names).reset_index(drop = True)

    #-------------------------------Fitting Model----------------------------
    # Make an instance of the Model
    pca = PCA(0.95, random_state = 1) #95% of the variance (information) is retained

    x1 = pca.fit_transform(X_train) #PCA based on train data
    x2 = pca.transform(X_test) #transform test data
    
    filename = '../Dashboard UI/Dashboard UI/model/pca.pkl'
    pickle.dump(pca, open(filename, 'wb')) #save PCA model
    
    #Colab use
    #filename = 'pca.pkl'
    #pickle.dump(pca, open(filename, 'wb')) #save PCA model
    #files.download('pca.pkl')
    
    #Graph to indicate information contains in each component
    #fir = plt.figure(figsize=(8,5))
    #sing_vals = np.arange(len(pca.components_)) + 1
    #plt.plot(sing_vals, pca.explained_variance_ratio_, 'ro-', linewidth=2)
    #plt.title('Scree Plot', fontsize = 20)
    #plt.xlabel('Principal Component', fontsize = 20)
    #plt.ylabel('Eigenvalue', fontsize = 20)
    #plt.xticks(fontsize=10)
    #plt.yticks(fontsize=10)
    #------------------------------------------------------------------------
    
    #convert to data frame
    X_train = pd.DataFrame(data = x1)
    X_test = pd.DataFrame(data = x2)
    #------------------------------------------------------------------------
    
    return X_train, X_test, y_train, y_test

# Gradient Boosting Regression

In [6]:
def gradient_boosting_regressor(X_train, X_test, y_train, y_test):
    
    #for reproducible result
    np.random.seed(2)
    
    #------------------------------Train Model-------------------------------
    #GBR model for randomised search
    #GBR = GradientBoostingRegressor(random_state=1)
    
    #parameter after first grid search
    #GBR = GradientBoostingRegressor(learning_rate = 0.1, 
                                    #n_estimators = 100,
                                    #subsample = 0.6,
                                    #random_state=1)
    
    #parameter after second grid search
    #GBR = GradientBoostingRegressor(learning_rate = 0.1, 
                                    #n_estimators = 100,
                                    #subsample = 0.6,
                                    #min_samples_split = 1400,
                                    #max_depth = 5,
                                    #random_state=1)
    
    #parameter after third grid search
    GBR = GradientBoostingRegressor(learning_rate = 0.1, 
                                    n_estimators = 100,
                                    subsample = 0.6,
                                    min_samples_split = 1400,
                                    max_depth = 5,
                                    min_samples_leaf = 1,
                                    random_state=1)

    #shrinks the contribution of each tree (0.1)
    learning_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
    #The number of boosting stages to perform (100)
    n_estimators = list(range(40, 200, 20))
    #The fraction of samples to be used for fitting the individual base learners
    subsample = [0.2, 0.4, 0.6, 0.8, 1.0]
    
    #minimum number of samples required to split an internal node (2)
    min_samples_split = list(range(1000,10000,400))
    #Maximum depth of the individual regression estimators (3)
    max_depth = list(range(1,10,2))
    #max_depth.append(None)
    
    #minimum number of samples required to be at a leaf node (1)
    min_samples_leaf = list([1])
    
    
    #param_grid = {'learning_rate': learning_rate,
                  #'n_estimators': n_estimators,
                  #'subsample': subsample,
                  #'min_samples_split': min_samples_split,
                  #'max_depth': max_depth, 
                  #'min_samples_leaf': min_samples_leaf}
    
    # Create the dictionary
    #first grid search
    #param_grid = {'learning_rate': learning_rate,
                  #'n_estimators': n_estimators,
                  #'subsample': subsample
                 #}
    #second grid search
    #param_grid = {'min_samples_split': min_samples_split,
                  #'max_depth': max_depth
                 #}
    #third grid search
    #param_grid = {'min_samples_leaf': min_samples_leaf}
    
    #K-fold cross validation
    #cv = KFold(n_splits=10, shuffle = True, random_state = 1)
    
    #grid search 
    #grid_search = GridSearchCV(GBR, param_grid = param_grid, 
                               #cv = cv, n_jobs=-1)
    
    #grid_search.fit(X_train, y_train.values.ravel()) #train model using grid search

    GBR.fit(X_train, y_train.values.ravel())

    #rand_search = RandomizedSearchCV(GBR, param_distributions = param_grid, 
                                     #cv = cv, n_jobs=-1, random_state=1)
    #rand_search.fit(X_train, y_train.values.ravel()) #train model using randomised search
    #------------------------------------------------------------------------
    
    #------------------------------Predict X_test----------------------------
    #y_pred_train = rand_search.predict(X_train) 
    #y_pred_test = rand_search.predict(X_test)
    
    #y_pred_train = grid_search.predict(X_train)
    #y_pred_test = grid_search.predict(X_test)
    
    y_pred_train = GBR.predict(X_train) #predict on train data
    y_pred_test = GBR.predict(X_test) #predict on tets data
    #------------------------------------------------------------------------
    
    #---------------------------------Accuracy-------------------------------
    # Use score method to get accuracy of model
    #accuracy_score = rand_search.score(X_test, y_test) #for randomised search
    #accuracy_score = grid_search.score(X_test, y_test) #for grid search
    accuracy_score = GBR.score(X_test, y_test)
    print('Accuracy of Gradient Boosting Regression on test set: {:.2f}'.format(accuracy_score))
    #------------------------------------------------------------------------
    
    #---------------------------------RMSE & R2------------------------------
    evaluate(y_train, y_pred_train, 'Train')
    evaluate(y_test, y_pred_test, 'Test')
    #------------------------------------------------------------------------

    #---------------------------------Best Param-----------------------------
    #print(rand_search.best_params_) #best parameters of randomised search
    #print(rand_search.best_score_) #best score of randomised search
    
    #print(grid_search.best_params_) #best parameters of randomised search
    #print(grid_search.best_score_) #best score of grid search
    #print('\n')
    #------------------------------------------------------------------------
    
    filename = '../Dashboard UI/Dashboard UI/model/GBR_model.sav'
    pickle.dump(GBR, open(filename, 'wb')) #save GBR model
    
    #Colab use
    #filename = 'GBR_model.sav'
    #pickle.dump(GBR, open(filename, 'wb')) #save GBR model
    #files.download('GBR_model.sav')

# Working

In [7]:
X_train, X_test, y_train, y_test = pca(train, test, train_y, test_y)

print(X_test.shape)

gradient_boosting_regressor(X_train, X_test, y_train, y_test)

(100, 10)
Accuracy of Gradient Boosting Regression on test set: 0.83
Train set RMSE:17.28698720544705, R2:0.827917948833811
Test set RMSE:17.3438063091837, R2:0.8258075668883855


{'learning_rate': 0.1, 'max_features': 'auto', 'n_estimators': 90, 'subsample': 1.0}
{'max_depth': 5, 'min_samples_split': 1400}
{'min_samples_leaf': 37}